In [1]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta
import hashlib
import numpy as np

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="yape_database",
    port=5434,
    user="gmborjasb",
    password="MomoYMoka"
)

faker = Faker('es_MX')
cur = conn.cursor()

In [3]:
# Ubicarse en un esquema
cur.execute("SET search_path TO yape_1_000;")
cur.execute("SHOW search_path;")
print(cur.fetchone())

('yape_1_000',)


In [4]:
# ==========================
# Cantidades
# ==========================
num_actors = 1000
num_persons = int(num_actors * 0.7)
num_companies = num_actors - num_persons

print(f"Personas: {num_persons}, Empresas: {num_companies}")

Personas: 700, Empresas: 300


In [5]:
# ==========================
# Insertar ActorYape
# ==========================
actor_ids = []

for _ in range(num_actors):
    cur.execute("INSERT INTO Actor_Yape DEFAULT VALUES RETURNING id_actor;")
    actor_ids.append(cur.fetchone()[0])

conn.commit()

In [6]:
# ==========================
# Dividir actores en grupos
# ==========================
person_ids = actor_ids[:num_persons]
company_ids = actor_ids[num_persons:]

In [7]:
# ==========================
# Insertar Personas
# ==========================

# Para evitar colisiones únicas generamos un set
used_dnis = set()


def generar_dni_unico():
    while True:
        dni = str(random.randint(70000000, 79999999))  # Rango realista
        if dni not in used_dnis:
            used_dnis.add(dni)
            return dni


for actor_id in person_ids:
    dni = generar_dni_unico()
    correo = faker.unique.email()

    cur.execute("""
        INSERT INTO Persona (id_actor, dni, correo)
        VALUES (%s, %s, %s);
    """, (actor_id, dni, correo))

# ==========================
# Insertar Empresas
# ==========================

used_rucs = set()


def generar_ruc_unico():
    while True:
        # 11 dígitos válidos
        ruc = str(random.randint(10000000000, 10999999999))
        if ruc not in used_rucs:
            used_rucs.add(ruc)
            return ruc


for actor_id in company_ids:
    ruc = generar_ruc_unico()
    razon_social = faker.company()
    rubro = faker.bs()
    nombre_comercial = razon_social.split()[0]

    cur.execute("""
        INSERT INTO Empresa (id_actor, ruc, razon_social, rubro_comercial, nombre_comercial)
        VALUES (%s, %s, %s, %s, %s);
    """, (actor_id, ruc, razon_social, rubro, nombre_comercial))
    
conn.commit()

In [8]:
# Cantidad EmpresaPequeñoNegocio
num_small_businesses = int(num_companies * 0.33)
# Cantidad EmpresaAE
num_ae_companies = int(num_companies * 0.33)
# Cantidad EmpresaServicios
num_service_companies = num_companies - num_small_businesses - num_ae_companies

# Subdivisión de empresas
small_business_ids = company_ids[:num_small_businesses]
ae_business_ids = company_ids[num_small_businesses:
                              num_small_businesses + num_ae_companies]
service_business_ids = company_ids[num_small_businesses + num_ae_companies:]

In [9]:
# =========================================
#   CREACIÓN DE EMPRESAS Y SUBCLASES
# =========================================

# -----------------------------------------
# Función: Empresa Pequeño Negocio
# -----------------------------------------


def insert_small_business(actor_id):
    dni_duenho = str(random.randint(70000000, 79999999))
    correo_contacto = faker.email()
    celular_contacto = str(random.randint(900000000, 989999999))

    cur.execute("""
        INSERT INTO Empresa_Pequeno_Negocio 
            (id_actor, dni_duenho, correo_contacto, celular_contacto)
        VALUES (%s, %s, %s, %s);
    """, (actor_id, dni_duenho, correo_contacto, celular_contacto))

# -----------------------------------------
# Función: Empresa Acceso Empresarial
# -----------------------------------------


def insert_ae(actor_id):
    cur.execute("""
        INSERT INTO Empresa_Acceso_Empresarial (id_actor)
        VALUES (%s);
    """, (actor_id,))

# -----------------------------------------
# Función: Empresa Servicios
# -----------------------------------------


def insert_service_company(actor_id):
    cur.execute("""
        INSERT INTO Empresa_Servicios (id_actor)
        VALUES (%s);
    """, (actor_id,))

# =========================================
#   INSERCIÓN FINAL SEGÚN TUS DISTRIBUCIONES
# =========================================

print("Insertando subclases de Empresa")


# 1. Empresa Pequeño Negocio
for actor_id in small_business_ids:
    insert_small_business(actor_id)

# 2. Empresa Acceso Empresarial
for actor_id in ae_business_ids:
    insert_ae(actor_id)

# 3. Empresa Servicios
for actor_id in service_business_ids:
    insert_service_company(actor_id)

conn.commit()

print("✔ Subclases de empresa generadas correctamente.")


Insertando subclases de Empresa
✔ Subclases de empresa generadas correctamente.


In [10]:
# ==========================================
# LISTA COMPLETA DE FUNCIONALIDADES
# (nombre, descripción)
# ==========================================
funcionalidades = [
    # --- Uso general ---
    ("Enviar dinero", "Permite transferir dinero entre billeteras Yape o redes interoperables."),
    ("Recibir dinero", "Registra y notifica la recepción de pagos."),
    ("Editar perfil", "Permite modificar los datos básicos del usuario."),
    ("Gestión de credenciales", "Registrar, actualizar o validar PIN o biometría."),
    ("Verificación de identidad", "Autenticación avanzada mediante DNI o biometría."),
    ("Consultar saldo", "Visualiza el saldo disponible en la billetera."),
    ("Consultar movimientos", "Muestra el historial de operaciones."),
    ("Bloqueo temporal de billetera", "Permite bloquear la billetera por seguridad."),

    # --- Interoperabilidad ---
    ("Enviar a otras billeteras", "Permite transferencias a Plin, Agora u otras redes."),
    ("Recibir de otras billeteras", "Permite recibir pagos de redes externas."),

    # --- Servicios ---
    ("Pago de servicios", "Permite efectuar pagos a empresas de servicios."),
    ("Notificación de pago de servicio", "Alerta generada al pagar un servicio."),

    # --- QR ---
    ("Pago con QR", "Permite pagar mediante un QR dinámico o estático."),
    ("Generar QR estático", "Genera un código QR permanente para cobros."),
    ("Generar QR dinámico", "Genera un QR empresarial con monto fijo."),

    # --- Empresarial ---
    ("Gestión de cuenta de recaudación",
     "Configurar la cuenta de recaudación empresarial."),
    ("Consulta de comisiones", "Consulta de comisiones aplicadas."),
    ("Gestión de cobros", "Permite generar solicitudes de cobro AE."),
    ("Reportes empresariales", "Proporciona reportes diarios/mensuales."),

    # --- Notificaciones ---
    ("Notificación de envío de dinero", "Aviso generado por envíos de dinero."),
    ("Notificación de recepción de dinero", "Aviso al recibir dinero."),
    ("Notificación de seguridad", "Alertas de fraude, cambios críticos o bloqueos."),

    # --- Analítica ---
    ("Estadísticas de transacciones", "Indicadores de actividad financiera."),
    ("Resumen mensual", "Resumen de operaciones del mes.")
]

# ==========================================
# INSERTAR FUNCIONALIDADES (si no existen)
# ==========================================
func_ids = {}

for nombre, desc in funcionalidades:
    cur.execute("""
        INSERT INTO funcionalidad (nombre, descripcion)
        VALUES (%s, %s)
        RETURNING id_funcionalidad;
    """, (nombre, desc))
    func_ids[nombre] = cur.fetchone()[0]

conn.commit()
print("✔ Funcionalidades insertadas correctamente.")

✔ Funcionalidades insertadas correctamente.


In [11]:
# ==========================================
# DEFINIR FUNCIONALIDADES POR TIPO DE ACTOR
# ==========================================

F_P = [
    "Enviar dinero", "Recibir dinero", "Editar perfil", "Gestión de credenciales",
    "Verificación de identidad", "Consultar saldo", "Consultar movimientos",
    "Enviar a otras billeteras", "Recibir de otras billeteras",
    "Pago de servicios", "Pago con QR", "Generar QR estático",
    "Notificación de envío de dinero", "Notificación de recepción de dinero",
    "Notificación de seguridad", "Resumen mensual"
]

F_EPN = [
    "Enviar dinero", "Recibir dinero", "Consultar saldo", "Consultar movimientos",
    "Pago con QR", "Generar QR estático",
    "Pago de servicios", "Notificación de recepción de dinero",
    "Gestión de cuenta de recaudación", "Consulta de comisiones",
    "Reportes empresariales", "Resumen mensual"
]

F_EAE = [
    "Recibir dinero", "Consultar saldo", "Consultar movimientos",
    "Generar QR dinámico", "Gestión de cobros",
    "Consulta de comisiones", "Reportes empresariales",
    "Notificación de recepción de dinero"
]

F_ES = [
    "Recibir dinero", "Consultar saldo", "Consultar movimientos",
    "Generar QR estático", "Reportes empresariales",
    "Pago de servicios", "Notificación de recepción de dinero"
]


# ==========================================
# FUNCIÓN PARA INSERTAR EN A_Funcionalidad
# ==========================================
def asignar(actor_ids, funciones):
    for actor_id in actor_ids:
        for f in funciones:
            cur.execute("""
                INSERT INTO a_funcionalidad (id_actor, id_funcionalidad)
                VALUES (%s, %s);
            """, (actor_id, func_ids[f]))


# ==========================================
# ASIGNAR FUNCIONALIDADES
# ==========================================

asignar(person_ids, F_P)
asignar(small_business_ids, F_EPN)
asignar(ae_business_ids, F_EAE)
asignar(service_business_ids, F_ES)

conn.commit()

print("✔ Funcionalidades asignadas correctamente.")

✔ Funcionalidades asignadas correctamente.


In [12]:
# ======================================================
# CREAR BILLETERA BASE
# ======================================================

def crear_billetera_base(id_actor, origen, interoperabilidad):
    celular = str(random.randint(900000000, 989999999))
    estado = random.choice(
        ["Activo", "Activo", "Activo", "Bloqueado", "Inactivo"])

    cur.execute("""
        INSERT INTO billetera_yape 
        (id_actor, celular, origen_billetera, estado, permite_interoperabilidad)
        VALUES (%s, %s, %s, %s, %s)
        RETURNING id_billetera;
    """, (id_actor, celular, origen, estado, interoperabilidad))

    return cur.fetchone()[0]

# ======================================================
# BilleteraPersona
# ======================================================

print("▶ Creando billeteras de personas…")

for id_actor in person_ids:
    id_billetera = crear_billetera_base(
        id_actor=id_actor,
        origen="Yape",
        interoperabilidad=True
    )

    metodo = random.choice(["DNI", "Facial", "Huella"])
    limite_diario = round(random.uniform(300, 1500), 0)
    limite_mensual = round(random.uniform(2000, 6000), 0)
    limite_op = round(random.uniform(50, 500), 0)
    nivel = random.choice(["Basico", "Avanzado"])

    cur.execute("""
        INSERT INTO billetera_persona
        (id_billetera, metodo_registro, limite_diario, limite_mensual_recaudacion,
         limite_por_operacion, bloqueado_fraude, nivel_verificacion)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, (id_billetera, metodo, limite_diario, limite_mensual, limite_op, False, nivel))


# ======================================================
# BilleteraEmpresa
# ======================================================

print("▶ Creando billeteras de empresas…")


def billetera_empresa(id_actor):
    id_billetera = crear_billetera_base(
        id_actor=id_actor,
        origen="Yape",
        interoperabilidad=False
    )

    cuenta = str(random.randint(10000000000, 99999999999))
    comision = round(random.uniform(0.02, 0.05), 4)
    limite_mensual = round(random.uniform(20000, 150000), 0)

    cur.execute("""
        INSERT INTO billetera_empresa
        (id_billetera, cuenta_recaudacion, tasa_comision, limite_mensual_recaudacion)
        VALUES (%s, %s, %s, %s);
    """, (id_billetera, cuenta, comision, limite_mensual))


# Asignación para cada tipo de empresa
for id_actor in small_business_ids:
    billetera_empresa(id_actor)

for id_actor in ae_business_ids:
    billetera_empresa(id_actor)

for id_actor in service_business_ids:
    billetera_empresa(id_actor)


# ======================================================
# BilleteraOtherPersona
# ======================================================

print("▶ Creando billeteras de otras redes (Plin, Tunki)…")

num_other = int (0.5 * num_actors)   # cantidad de billeteras_other_persona

for _ in range(num_other):
    id_billetera = crear_billetera_base(
        id_actor=None,                 # No referencia a ActorYape
        origen=random.choice(["Plin", "Tunki", "Agora"]),
        interoperabilidad=True
    )

    id_ext = faker.uuid4()
    nombre = faker.name()

    cur.execute("""
        INSERT INTO billetera_other_persona
        (id_billetera, id_externo, nombre_externo)
        VALUES (%s, %s, %s);
    """, (id_billetera, id_ext, nombre))

conn.commit()

print("✔  Billeteras generadas correctamente.")

▶ Creando billeteras de personas…
▶ Creando billeteras de empresas…
▶ Creando billeteras de otras redes (Plin, Tunki)…
✔  Billeteras generadas correctamente.


In [ ]:
# ==========================================================
# OBTENER TODAS LAS BILLETERAS
# ==========================================================
cur.execute("SELECT id_billetera FROM billetera_yape;")
billeteras = [r[0] for r in cur.fetchall()]

print(f"Total billeteras encontradas: {len(billeteras)}")


# ==========================================================
# TIPOS DE CREDENCIAL PERMITIDOS
# ==========================================================
TIPOS = ["PIN", "Huella", "Facial", "Patron"]


# ==========================================================
# FUNCIÓN PARA CREAR HASH REALISTA
# ==========================================================
def generar_hash(valor):
    return hashlib.sha256(valor.encode()).hexdigest()


# ==========================================================
# CREAR CREDENCIALES PARA CADA BILLETERA
# ==========================================================
print("▶ Generando credenciales…")

for id_billetera in billeteras:

    # Cantidad de credenciales por billetera
    num_creds = random.randint(1, 3)

    # Elegir 1 activa y otras inactivas
    tipos_seleccionados = random.sample(TIPOS, num_creds)
    tipo_activo = random.choice(tipos_seleccionados)

    for idx, tipo in enumerate(tipos_seleccionados, start=1):

        # hash_value realista
        raw_value = faker.password() + tipo
        hash_val = generar_hash(raw_value)

        # Estado → solo una activa
        estado = "Activa" if tipo == tipo_activo else random.choice(
            ["Expirada", "Bloqueada"])

        fecha_creacion = faker.date_time_between(
            start_date="-1y", end_date="now")
        fecha_expiracion = None

        if estado != "Activa":
            fecha_expiracion = fecha_creacion + \
                timedelta(days=random.randint(30, 300))

        cur.execute("""
            INSERT INTO credencial 
            (id_billetera, id_credencial, tipo, hash_valor, fecha_creacion, fecha_expiracion, estado)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (
            id_billetera,
            idx,  # id_credencial por billetera
            tipo,
            hash_val,
            fecha_creacion,
            fecha_expiracion,
            estado
        ))

conn.commit()

print("✔ Credenciales generadas correctamente.")

Total billeteras encontradas: 1500
▶ Generando credenciales…
✔ Credenciales generadas correctamente.


In [14]:
# =========================================================
# OBTENER BILLETERAS PERMITIDAS PARA QR ESTÁTICO
# =========================================================

# BilleteraPersona
cur.execute("""
    SELECT id_billetera
    FROM billetera_persona;
""")
billeteras_persona = [r[0] for r in cur.fetchall()]

# EmpresaPequeñoNegocio → BilleteraEmpresa asociada
cur.execute("""
    SELECT be.id_billetera
    FROM billetera_empresa be
    JOIN empresa_pequeno_negocio epn ON be.id_billetera IN (
        SELECT id_billetera FROM billetera_yape WHERE id_actor = epn.id_actor
    );
""")
billeteras_epn = [r[0] for r in cur.fetchall()]

# Billeteras que SI pueden tener QR estático
billeteras_estaticas_permitidas = billeteras_persona + billeteras_epn

# =========================================================
# DEFINIR TOTAL DE QR A GENERAR
# =========================================================
num_qr_estaticos = len(billeteras_estaticas_permitidas)  # 30% estáticos
total_qr = int( 10 / 3 * num_qr_estaticos )   

print("Total QR:", total_qr)
print("QR estáticos:", num_qr_estaticos)

# =========================================================
# CREAR QR GENERALES
# =========================================================
qr_ids = []

for _ in range(total_qr):
    codigo_qr = faker.uuid4()
    activo = True  # por defecto

    cur.execute("""
        INSERT INTO qr (codigo_qr, activo)
        VALUES (%s, %s)
        RETURNING id_qr;
    """, (codigo_qr, activo))

    qr_ids.append(cur.fetchone()[0])

conn.commit()

# =========================================================
# CREAR QR ESTÁTICOS (30% del total)
# =========================================================

# QR que serán estáticos
qr_estaticos_ids = random.sample(qr_ids, num_qr_estaticos)

# Billeteras permitidas donde estarán los QR activos
billeteras_destino = billeteras_estaticas_permitidas.copy()

random.shuffle(billeteras_destino)

index = 0

for id_qr in qr_estaticos_ids:
    # Si ya no quedan billeteras activas → QR será inactivo
    if index < len(billeteras_destino):
        id_billetera = billeteras_destino[index]
        activo = True
    else:
        # QR estático inactivo sin billetera válida
        id_billetera = random.choice(billeteras_estaticas_permitidas)
        activo = False

    index += 1

    cur.execute("""
        INSERT INTO qr_estatico (id_qr, id_billetera, activo)
        VALUES (%s, %s, %s);
    """, (id_qr, id_billetera, activo))

conn.commit()

print("✔ QR y QR_Estatico generados correctamente.")

Total QR: 2663
QR estáticos: 799
✔ QR y QR_Estatico generados correctamente.


In [15]:
# ==========================
# HELPERS
# ==========================

now = datetime.now()


def random_datetime_next_year():
    """Fecha aleatoria entre ahora y un año después."""
    # días y segundos aleatorios
    delta = timedelta(
        days=random.uniform(0, 365),
        seconds=random.uniform(0, 24 * 3600)
    )
    return now + delta


def random_estado():
    """80% Exitosa, resto repartido Pendiente/Fallida/Revertida."""
    r = random.random()
    if r < 0.8:
        return "Exitosa"
    elif r < 0.8 + (0.2 / 3):
        return "Pendiente"
    elif r < 0.8 + (2 * 0.2 / 3):
        return "Fallida"
    else:
        return "Revertida"


def random_tipo_destino():
    """80% Yape, 20% repartido entre Plin/Tunki/Agora/Otro."""
    r = random.random()
    if r < 0.8:
        return "Yape"
    else:
        return random.choice(["Plin", "Tunki", "Agora", "Otro"])


# ==========================
# FUNCIÓN PARA GENERAR MONTOS CON DISTRIBUCIÓN NORMAL
# ==========================
def generar_monto_normal(media=87, desviacion=30, minimo=1, maximo=500):
    """
    Genera un monto con distribución normal.
    
    Args:
        media: valor promedio (87)
        desviacion: desviación estándar (ajusta según dispersión deseada)
        minimo: monto mínimo permitido
        maximo: monto máximo permitido
    
    Returns:
        float: monto redondeado a 2 decimales
    """
    while True:
        monto = np.random.normal(media, desviacion)
        if minimo <= monto <= maximo:
            return round(monto, 2)


# ==========================
# CONTAR BILLETERAS
# ==========================
cur.execute("SELECT COUNT(*) FROM Billetera_Yape;")
num_billeteras = cur.fetchone()[0]
print(f"Billeteras encontradas: {num_billeteras}")

if num_billeteras == 0:
    print("No hay billeteras, no se pueden generar transacciones.")
else:
    total_tx = num_billeteras * 10

    tx_persona = int(total_tx * 0.8)
    restantes = total_tx - tx_persona
    tx_servicios = restantes // 2
    tx_ae = restantes - tx_servicios

    print(f"Transacciones totales: {total_tx}")
    print(f" - Persona:   {tx_persona}")
    print(f" - Servicios: {tx_servicios}")
    print(f" - AE:        {tx_ae}")

    # ==========================
    # GENERAR TRANSACCION_PERSONA (80%)
    # ==========================
    print("▶ Generando Transaccion_Persona…")

    numero_operacion_counter = 1  # para asegurar unicidad

    for _ in range(tx_persona):
        fecha_hora = random_datetime_next_year()
        estado = random_estado()
        monto = generar_monto_normal(
            media = 87, desviacion = 30)  # montos razonables
        numero_operacion = f"OP{numero_operacion_counter:010d}"
        numero_operacion_counter += 1

        # Insert base Transaccion_Yape
        cur.execute("""
            INSERT INTO Transaccion_Yape (fecha_hora, estado, monto, numero_operacion)
            VALUES (%s, %s, %s, %s)
            RETURNING id_transaccion;
        """, (fecha_hora, estado, monto, numero_operacion))
        id_tx = cur.fetchone()[0]

        # Datos específicos de Persona
        codigo_seguridad = str(random.randint(0, 9999)).zfill(4)
        tipo_destino = random_tipo_destino()
        numero_receptor = str(random.randint(
            900000000, 999999999))  # 9 dígitos
        nombre_receptor = faker.name()
        nombre_emisor = faker.name()

        cur.execute("""
            INSERT INTO Transaccion_Persona
            (id_transaccion, codigo_seguridad, tipo_destino, numero_receptor, nombre_receptor, nombre_emisor)
            VALUES (%s, %s, %s, %s, %s, %s);
        """, (id_tx, codigo_seguridad, tipo_destino, numero_receptor, nombre_receptor, nombre_emisor))

    conn.commit()
    print("✔ Transaccion_Persona generadas.")

    # ==========================
    # GENERAR TRANSACCION_SERVICIOS (10%)
    # ==========================
    print("▶ Generando Transaccion_Servicios…")

    tipos_servicio = ["Agua", "Luz", "Telefono", "Internet", "Cable", "Otros"]

    for _ in range(tx_servicios):
        fecha_hora = random_datetime_next_year()
        estado = random_estado()
        monto = generar_monto_normal(media=100, desviacion=30, minimo = 10, maximo = 3000)
        numero_operacion = f"OP{numero_operacion_counter:010d}"
        numero_operacion_counter += 1

        # Base
        cur.execute("""
            INSERT INTO Transaccion_Yape (fecha_hora, estado, monto, numero_operacion)
            VALUES (%s, %s, %s, %s)
            RETURNING id_transaccion;
        """, (fecha_hora, estado, monto, numero_operacion))
        id_tx = cur.fetchone()[0]

        codigo = f"SERV{random.randint(100000, 999999)}"
        descripcion = faker.sentence(nb_words=6)
        tipo_operacion_servicio = random.choice(tipos_servicio)

        cur.execute("""
            INSERT INTO Transaccion_Servicios
            (id_transaccion, codigo, descripcion, tipo_operacion_servicio)
            VALUES (%s, %s, %s, %s);
        """, (id_tx, codigo, descripcion, tipo_operacion_servicio))

    conn.commit()
    print("✔ Transaccion_Servicios generadas.")

    # ==========================
    # GENERAR TRANSACCION_EMPRESA_AE (10%) + QR_DINAMICO
    # ==========================
    print("▶ Generando Transaccion_Empresa_AE y QR_Dinamico…")

    for _ in range(tx_ae):
        fecha_hora = random_datetime_next_year()
        estado = random_estado()
        monto = generar_monto_normal(media=87, desviacion=40, minimo = 10, maximo = 3000)
        numero_operacion = f"OP{numero_operacion_counter:010d}"
        numero_operacion_counter += 1

        # Base Transaccion_Yape
        cur.execute("""
            INSERT INTO Transaccion_Yape (fecha_hora, estado, monto, numero_operacion)
            VALUES (%s, %s, %s, %s)
            RETURNING id_transaccion;
        """, (fecha_hora, estado, monto, numero_operacion))
        id_tx = cur.fetchone()[0]

        # Crear QR base + dinámico asociado
        codigo_qr = f"QRAE{random.randint(1000000, 9999999)}"
        fecha_creacion_qr = fecha_hora.date()
        activo_qr = True

        # Insert en QR y obtener id_qr
        cur.execute("""
            INSERT INTO QR (codigo_qr, fecha_creacion, activo)
            VALUES (%s, %s, %s)
            RETURNING id_qr;
        """, (codigo_qr, fecha_creacion_qr, activo_qr))
        id_qr = cur.fetchone()[0]

        # Insert en QR_Dinamico
        tiempo_validez = random.randint(5, 60)  # minutos
        cur.execute("""
            INSERT INTO QR_Dinamico (id_qr, monto_fijo, tiempo_validez)
            VALUES (%s, %s, %s);
        """, (id_qr, monto, tiempo_validez))

        # Insert en Transaccion_Empresa_AE
        fecha_confirmacion = fecha_hora.date()
        descripcion_ae = faker.sentence(nb_words=8)

        cur.execute("""
            INSERT INTO Transaccion_Empresa_AE
            (id_transaccion, id_qr, fecha_confirmacion, descripcion)
            VALUES (%s, %s, %s, %s);
        """, (id_tx, id_qr, fecha_confirmacion, descripcion_ae))

    conn.commit()

    print("✔ Transaccion_Empresa_AE y QR_Dinamico generados.")
    print("✅ Todas las transacciones se generaron correctamente.")

Billeteras encontradas: 1500
Transacciones totales: 15000
 - Persona:   12000
 - Servicios: 1500
 - AE:        1500
▶ Generando Transaccion_Persona…
✔ Transaccion_Persona generadas.
▶ Generando Transaccion_Servicios…
✔ Transaccion_Servicios generadas.
▶ Generando Transaccion_Empresa_AE y QR_Dinamico…
✔ Transaccion_Empresa_AE y QR_Dinamico generados.
✅ Todas las transacciones se generaron correctamente.


In [ ]:
# =====================================================
# OBTENER BILLETERAS POR TIPO
# =====================================================

# BilleteraPersona
cur.execute("SELECT id_billetera FROM billetera_persona;")
billetera_persona = [r[0] for r in cur.fetchall()]

# BilleteraOtherPersona
cur.execute("SELECT id_billetera FROM billetera_other_persona;")
billetera_other = [r[0] for r in cur.fetchall()]

# BilleteraEmpresa asociada a EmpresaPequeñoNegocio
cur.execute("""
    SELECT by.id_billetera
    FROM billetera_yape by
    JOIN billetera_empresa be ON by.id_billetera = be.id_billetera
    JOIN empresa_pequeno_negocio epn ON by.id_actor = epn.id_actor;
""")
billetera_epn = [r[0] for r in cur.fetchall()]

# BilleteraEmpresa asociada a EmpresaServicios
cur.execute("""
    SELECT by.id_billetera
    FROM billetera_yape by
    JOIN billetera_empresa be ON by.id_billetera = be.id_billetera
    JOIN empresa_servicios es ON by.id_actor = es.id_actor;
""")
billetera_es = [r[0] for r in cur.fetchall()]

# BilleteraEmpresa asociada a EmpresaAE
cur.execute("""
    SELECT by.id_billetera
    FROM billetera_yape by
    JOIN billetera_empresa be ON by.id_billetera = be.id_billetera
    JOIN empresa_acceso_empresarial ae ON by.id_actor = ae.id_actor;
""")
billetera_ae = [r[0] for r in cur.fetchall()]

# =====================================================
# OBTENER TRANSACCIONES POR TIPO
# =====================================================

cur.execute("SELECT id_transaccion FROM transaccion_persona;")
tx_persona = [r[0] for r in cur.fetchall()]

cur.execute("SELECT id_transaccion FROM transaccion_servicios;")
tx_serv = [r[0] for r in cur.fetchall()]

cur.execute("SELECT id_transaccion FROM transaccion_empresa_ae;")
tx_ae = [r[0] for r in cur.fetchall()]


# =====================================================
# FUNCIÓN HELPER PARA INSERCIÓN SEGURA
# =====================================================

def insertar_operacion(id_tx, emisor, receptor):
    if emisor == receptor:
        # evitar violación de chk_op_distintos
        return

    cur.execute("""
        INSERT INTO operacion_yape
        (id_transaccion, id_billetera_emisor, id_billetera_receptor)
        VALUES (%s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (id_tx, emisor, receptor))


# =====================================================
# TRANSACCION_PERSONA (80%)
# =====================================================

print("▶ Insertando Operacion_Yape para Transaccion_Persona…")

emisores_persona = billetera_persona + billetera_other + billetera_epn
receptores_persona = emisores_persona  # mismo conjunto

for id_tx in tx_persona:
    emisor = random.choice(emisores_persona)
    receptor = random.choice(receptores_persona)

    # evitar emisor == receptor
    while receptor == emisor:
        receptor = random.choice(receptores_persona)

    insertar_operacion(id_tx, emisor, receptor)


# =====================================================
# TRANSACCION_SERVICIOS (10%)
# =====================================================

print("▶ Insertando Operacion_Yape para Transaccion_Servicios…")

# Emisor: SOLO billeteras persona
# Receptor: SOLO billetera empresa de empresa_servicios

for id_tx in tx_serv:
    emisor = random.choice(billetera_persona)
    receptor = random.choice(billetera_es)

    insertar_operacion(id_tx, emisor, receptor)


# =====================================================
# TRANSACCION_AE (10%)
# =====================================================

print("▶ Insertando Operacion_Yape para Transaccion_Empresa_AE…")

# Emisor: SOLO billetera persona
# Receptor: SOLO billetera empresa asociada a AE

for id_tx in tx_ae:
    emisor = random.choice(billetera_persona)
    receptor = random.choice(billetera_ae)

    insertar_operacion(id_tx, emisor, receptor)


# =====================================================
# FINALIZAR
# =====================================================

conn.commit()

print("✔ Operacion_Yape generada correctamente para TODOS los tipos.")

▶ Insertando Operacion_Yape para Transaccion_Persona…
▶ Insertando Operacion_Yape para Transaccion_Servicios…
▶ Insertando Operacion_Yape para Transaccion_Empresa_AE…
✔ Operacion_Yape generada correctamente para TODOS los tipos.
🎉 Modelo semántico respetado 100%.


In [17]:
# ==========================================
# OBTENER TRANSACCIONES Y MONTOS
# ==========================================
cur.execute("""
    SELECT id_transaccion, estado, monto
    FROM transaccion_yape;
""")
transacciones = cur.fetchall()  # [(id_tx, estado, monto), ...]


# ==========================================
# FUNCIÓN PARA CREAR NOTIFICACIÓN BASE
# ==========================================
def crear_notificacion(mensaje, estado='Enviada', canal='App'):
    cur.execute("""
        INSERT INTO notificacion (mensaje, estado, canal)
        VALUES (%s, %s, %s)
        RETURNING id_notificacion;
    """, (mensaje, estado, canal))
    return cur.fetchone()[0]


# ==========================================
# NOTIFICACIONES DE TRANSACCIONES
# ==========================================
print("▶ Generando Notificacion_Transaccion…")

for id_tx, estado_tx, monto in transacciones:

    # Mensaje base
    mensaje = f"Transacción {id_tx} procesada."

    id_notif = crear_notificacion(mensaje)

    # Determinar resultado
    if estado_tx == "Exitosa":
        resultado = "Exitosa"
    else:
        resultado = "Fallida"

    # Determinar tipo de operación
    # P2P, P2B, interoperables → "Envio" o "Recepcion"
    cur.execute("""
        SELECT id_billetera_emisor, id_billetera_receptor
        FROM operacion_yape
        WHERE id_transaccion = %s;
    """, (id_tx,))
    row = cur.fetchone()

    if row:
        id_emisor, id_receptor = row

        # Determinar tipo
        # PAGO SERVICIOS
        cur.execute("""
            SELECT 1 FROM transaccion_servicios WHERE id_transaccion = %s;
        """, (id_tx,))
        if cur.fetchone():
            tipo_operacion = "PagoServicio"

        # EMPRESA AE
        else:
            cur.execute("""
                SELECT 1 FROM transaccion_empresa_ae WHERE id_transaccion = %s;
            """, (id_tx,))
            if cur.fetchone():
                tipo_operacion = "CobroAE"
            else:
                # P2P / interoperable
                tipo_operacion = "Envio"  # por simplicidad

    else:
        # Si no tiene operacion_yape (no debería)
        tipo_operacion = "Envio"

    # Insert final
    cur.execute("""
        INSERT INTO Notificacion_Transaccion
            (id_notificacion, id_transaccion, monto, resultado, tipo_operacion)
        VALUES (%s, %s, %s, %s, %s);
    """, (id_notif, id_tx, monto, resultado, tipo_operacion))


# ==========================================
# NOTIFICACIONES DE BILLETERA (10%)
# ==========================================
print("▶ Generando Notificacion_Billetera (10%)…")

cur.execute("SELECT id_billetera FROM billetera_yape;")
todas_billeteras = [r[0] for r in cur.fetchall()]

num_alertas = max(1, int(len(todas_billeteras) * 0.10))
billeteras_alertadas = random.sample(todas_billeteras, num_alertas)

for id_b in billeteras_alertadas:
    mensaje = f"Billetera {id_b}: límite diario superado."
    id_notif = crear_notificacion(mensaje)

    cur.execute("""
        INSERT INTO Notificacion_Billetera
            (id_notificacion, id_billetera, tipo_alerta)
        VALUES (%s, %s, 'LimiteSuperado');
    """, (id_notif, id_b))


# ==========================================
# FINALIZAR
# ==========================================
conn.commit()

print("✔ Notificaciones generadas correctamente.")

▶ Generando Notificacion_Transaccion…
▶ Generando Notificacion_Billetera (10%)…
✔ Notificaciones generadas correctamente.


In [18]:
cur.close()
conn.close()